In [1]:
import pandas as pd
import os
import numpy as np
import numpy.ma as ma
import glob
from netCDF4 import Dataset,date2num,num2date
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import xlsxwriter

In [2]:
def extract_1(Var,min_index_lat,min_index_lon):
    cycle=np.array([Var[min_index_lat,min_index_lon],Var[min_index_lat+1,min_index_lon],
                    Var[min_index_lat-1,min_index_lon],Var[min_index_lat,min_index_lon+1],
                    Var[min_index_lat,min_index_lon-1]])
    cycle[cycle==0]=np.nan
    a=round(np.nanmean(cycle),0)
    return a
def extract_2(Var,min_index_lat,min_index_lon):
    cycle=np.array([Var[min_index_lat,min_index_lon],Var[min_index_lat+1,min_index_lon],
                    Var[min_index_lat-1,min_index_lon],Var[min_index_lat,min_index_lon+1],
                    Var[min_index_lat,min_index_lon-1]])
    cycle[cycle==0]=np.nan
    value=np.nanmean(cycle)
    return value

In [3]:
#read informations of gauge stations
stations=pd.read_excel(pd.ExcelFile("C:/Users/Admin/OneDrive/Desktop/gauges2.xlsx"),sheet_name='PA6')
#Choose a Case: Embedded/Squall line/Full stratiform
#Testing-Embedded:1,2,8,9,10,12,13,14,15/Full stratiform:1,6,7,8,10,12,13,14,15/Squall line:2,6,7,9,10,12,13,14,15
Case='General.xlsx'
#Choose events of a case
Case_dic={Case:[str(i) for i in[*range(1,28,1)]]}
frames_Z_ZDR_KDP=[]
#sheets=[]
for r in Case_dic[Case]:
    #make gauges dataframe at each events
    df_gauges=pd.read_excel('E:/data/Gauges/Finall/PA6/'+Case,sheet_name=r)
    #make dataframes for each dual-pol variables
    linkNC='D:/data/SVM_netCDF/'+Case[:-5]+'/'+r+'/'
    df_CS_SVM=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_Zh=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_ZDR=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_KDP=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    n=0
    for file_name in os.listdir(linkNC):
        data=Dataset(linkNC+file_name,'r') # at 1 time
        Zh=np.ma.power(10.,0.1*data.variables['Zh_1.5km'][0,:,:]).filled(fill_value=0.0)
        ZDR=np.ma.power(10.,0.1*data.variables['ZDR_1.5km'][0,:,:]).filled(fill_value=0.0)
        KDP=data.variables['KDP_1.5km'][0,:,:].filled(fill_value=0.0)
        for index,row in stations.iterrows():
            locations=row['Tên Trạm']
            location_lat=row['Latitude']
            location_lon=row['Longitude']
            lat=data.variables['latitude'][:]
            lon=data.variables['longitude'][:]
            sq_diff_lat=(lat-location_lat)**2
            sq_diff_lon=(lon-location_lon)**2
            min_index_lat=sq_diff_lat.argmin()
            min_index_lon=sq_diff_lon.argmin()
            df_CS_SVM.loc[[locations],list(df_gauges.columns[3:])[n]]=data.variables['CS_SVM'][0,min_index_lat,min_index_lon].filled(fill_value=0.0)
            df_Zh.loc[[locations],list(df_gauges.columns[3:])[n]]=np.log(Zh[min_index_lat,min_index_lon])
            df_ZDR.loc[[locations],list(df_gauges.columns[3:])[n]]=np.log(ZDR[min_index_lat,min_index_lon])
            df_KDP.loc[[locations],list(df_gauges.columns[3:])[n]]=np.log(KDP[min_index_lat,min_index_lon])
        n=n+1
    df=pd.DataFrame(data={  
    'Stations': list(df_gauges['Trạm Đo'][:])*df_gauges.columns[3:].size,
    'times': list(np.repeat(df_gauges.columns[3:],df_gauges['Trạm Đo'].size)),
    'SC_SVM':list(pd.Series(df_CS_SVM.iloc[:,:].values.ravel('F'))),
    'R_10p':list(np.log(pd.Series(df_gauges.iloc[:,3:].values.ravel('F'))*6)),
    'Z_1500m':list(pd.Series(df_Zh.iloc[:,:].values.ravel('F'))),
    'ZDR_1500m':list(pd.Series(df_ZDR.iloc[:,:].values.ravel('F'))),
    'KDP_1500m':list(pd.Series(df_KDP.iloc[:,:].values.ravel('F')))  
    })
    df['KDP_1500m'].replace([np.inf, -np.inf], 999, inplace=True)
    frames_Z_ZDR_KDP.append(df)
    print(r)
result = pd.concat(frames_Z_ZDR_KDP,ignore_index=True)
df_Stratiform_Z_ZDR=result.loc[(result.R_10p>0)&(result.SC_SVM==1)].drop(['KDP_1500m'], axis=1)
df_Convection_Z_ZDR=result.loc[(result.R_10p>0)&(result.SC_SVM==2)].drop(['KDP_1500m'], axis=1)
df_Stratiform_Z_ZDR_KDP=result.loc[(result.R_10p>0)&(result.SC_SVM==1)&(result.KDP_1500m!=999)]
df_Convection_Z_ZDR_KDP=result.loc[(result.R_10p>0)&(result.SC_SVM==2)&(result.KDP_1500m!=999)]
df_Z=result.loc[(result.R_10p>0)]
zz=[df_Stratiform_Z_ZDR,df_Convection_Z_ZDR,df_Stratiform_Z_ZDR_KDP,df_Convection_Z_ZDR_KDP,df_Z]
sheets=['Stratiform_Z_ZDR','Convection_Z_ZDR','Stratiform_Z_ZDR_KDP','Convection_Z_ZDR_KDP','Z']
out_path = r"E:/data/QPE/at_stations_PA6/"+Case 
writer = pd.ExcelWriter(out_path,engine='xlsxwriter')   
for dataframe, sheet in zip(zz, sheets):
    dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0,index=False)   
writer.save()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [7]:
df_Stratiform_Z_ZDR=result.loc[(result.R_10p>0)&(result.SC_SVM==1)].drop(['KDP_1500m'], axis=1)
df_Convection_Z_ZDR=result.loc[(result.R_10p>0)&(result.SC_SVM==2)].drop(['KDP_1500m'], axis=1)
df_Stratiform_Z_ZDR_KDP=result.loc[(result.R_10p>0)&(result.SC_SVM==1)&(result.KDP_1500m!=999)]
df_Convection_Z_ZDR_KDP=result.loc[(result.R_10p>0)&(result.SC_SVM==2)&(result.KDP_1500m!=999)]
df_Z=result.loc[(result.R_10p>0)]
zz=[df_Stratiform_Z_ZDR,df_Convection_Z_ZDR,df_Stratiform_Z_ZDR_KDP,df_Convection_Z_ZDR_KDP,df_Z]
sheets=['Stratiform_Z_ZDR','Convection_Z_ZDR','Stratiform_Z_ZDR_KDP','Convection_Z_ZDR_KDP','Z']
out_path = r"E:/data/QPE/at_stations_PA6/"+Case 
writer = pd.ExcelWriter(out_path,engine='xlsxwriter')   
for dataframe, sheet in zip(zz, sheets):
    dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0,index=False)   
writer.save()